In [1]:
# Reformatação com indexação hierárquica

In [2]:
# Stack --> faz a rotação ou o pivoteamento das colunas dos dados para as linhas
# Unstack --> faz o pivoteamento das linhas para as colunas

In [4]:
import numpy as np
import pandas as pd

In [5]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                   index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [9]:
result = data.stack() # pivoteamento das colunas para as linhas - sempre verifica se as colunas tem quantos index
# exemplo o one tem ohio e colorado, two também e por fim o three também

In [10]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [11]:
result.unstack() # reoorganiza de volta

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [12]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [13]:
result.unstack('state') # reorganiza com o state virando colunas

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [14]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [15]:
s1

a    0
b    1
c    2
d    3
dtype: int64

In [16]:
s2

c    4
d    5
e    6
dtype: int64

In [17]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [18]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [19]:
data2.unstack() # elementos da coluna que contém a, b, c, e d.. vira colunas, lembrando que aparece os nan

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [20]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [22]:
data2.unstack().stack(dropna=False) # aparece os nan em ambos por conta que não tem lista com elementos correspondentes

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [27]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [32]:
df = pd.DataFrame({'left': result})
df

left
state    number      
Ohio     one        0
         two        1
         three      2
Colorado one        3
         two        4
         three      5

In [34]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side')) # o right e left do dataframe concatena
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [38]:
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [39]:
df.unstack('state') # o parametro é para qual coluna que vai virar index APENAS

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [42]:
df.unstack('state').stack('side') # transforma a columns em linhas

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [43]:
# Fazendo o pivoteamento de um formato "longo" para um formato "largo"

In [47]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [53]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [54]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

In [57]:
data = data.reindex(columns=columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [61]:
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [62]:
data.index = periods.to_timestamp('D', 'end')
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [64]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [66]:
ldata[:10] # formato longo

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [68]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [70]:
ldata['value2'] = np.random.randn(len(ldata))

In [71]:
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,-1.783788
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.022243
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.821658
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.614607
4,1959-06-30 23:59:59.999999999,infl,2.340,1.972900
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.632285
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.557767
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.234069
8,1959-09-30 23:59:59.999999999,unemp,5.300,-1.450905
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.507577


In [72]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.022243 -1.783788   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.972900 -0.614607   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.234069 -0.557767   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.304826  0.507577   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.149659  1.244480   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.821658  
1959-06-30 23:59:59.999999999 -0.632285  
1959-09-30 23:59:59.999999999 -1.450905  
1959-12-31 23:59:59.999999999 -0.666343  
1960-03-31 23:59:59.999999999  0.865842

In [74]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.022243 -1.783788   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  1.972900 -0.614607   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.234069 -0.557767   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -1.304826  0.507577   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.149659  1.244480   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -0.451168 -1.247432   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6  0.112708 -1.486646   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.821658  
1959-06-30 23:59:59.999999999 -0.632285  
1959-09-30 23:59:59.999999999 -1.450905  
1959-12-31 23:59:59.999999999 -0.666343  
1960-03-31 23:59:59.999999999  0.865842  
1960-06-30 23:59:59.999999999  0.169981  
1960-09-30 23:59:59.999999999  0.051294

In [75]:
# Pivoteamento do formato "largo" para o formato "longo"

In [76]:
# operação inversa de pivot para DataFrame é pandas.melt

In [77]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                  'A': [1, 2, 3],
                  'B': [4, 5, 6],
                  'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [79]:
melted = pd.melt(df, ['key']) # o key é o indicador, ou sejá vai gerar várias keys com as variáveis divididas
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [81]:
reshaped = melted.pivot('key', 'variable', 'value') # reformatar e obter o layout original
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [82]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [85]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [86]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [87]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [89]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [90]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [93]:
pd.melt(df, ['key'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9
